In [12]:
# generates .csv files used to plot the maps in datawrapper

In [13]:
# import packages
import pandas as pd
import numpy as np
import utils.utils as u

In [14]:
# generates csv for the voluntary insurance adoption rates 
avg_tot_vol_ins_pur = pd.read_csv('../bigquery/data/avg_tot_vol_ins_pur.csv', dtype={'countyCode': str})
households = u.get_demographics([('Housing', 'Households')], name=True, state=True)
households['county_state'] = [c + ', ' + s if s=='AK' or s=='LA' else c + ' County, ' + s for c, s in zip(households.County, households.State)]
adopt_rate = avg_tot_vol_ins_pur[['countyCode', 'propertyState', 'year', 'buildingType', 'tot_pol_cnt']]
adopt_rate = adopt_rate[(adopt_rate.year==2021) & (adopt_rate.buildingType.str.contains('Residential'))].reset_index(drop=True)
gb_adopt = adopt_rate.groupby('countyCode')['tot_pol_cnt'].sum().reset_index()
gb_adopt2 = pd.merge(left=gb_adopt,
                     right=households[['CountyFIPS', 'County', 'State', 'county_state','Households']],
                     left_on='countyCode',
                     right_on='CountyFIPS',
                     how='inner'
                    )
gb_adopt2 = gb_adopt2[['countyCode', 'County', 'State', 'county_state', 'tot_pol_cnt', 'Households']]
gb_adopt2['adopt_rate'] = round((gb_adopt2['tot_pol_cnt']/gb_adopt2['Households'])*100,1)
gb_adopt2.to_csv('../data/adoption_rates.csv')

c:\Users\pacha\Dropbox\datascience\projects\florita\notebooks\utils\utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column[1]] = [by_code[fips][column[0]][column[1]] for fips in FipsDF.CountyFIPS]
c:\Users\pacha\Dropbox\datascience\projects\florita\notebooks\utils\utils.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['County'] =  [by_code[fips]['CountyFull'] for fips in FipsDF.CountyFIPS]
c:\Users\pacha\Dropbox\datascience\projects\florita\notebooks\utils\utils.py:32: SettingWithCopyWa

In [17]:
# generates dataframe with county names, fips, insurance rates per policy, median house prices
avg_tot_vol_ins_pur = pd.read_csv('../bigquery/data/avg_tot_vol_ins_pur.csv', dtype={'countyCode': str})
household_value = u.get_demographics([('Housing', 'Median Value of Owner-Occupied Units')], name=True, state=True)
household_value.columns = household_value.columns.str.replace(' ', '_').str.replace('-', '_')
household_value['county_state'] = [c + ', ' + s if s=='AK' or s=='LA' else c + ' County, ' + s for c, s in zip(household_value.County, household_value.State)]
ins_rate_vol = avg_tot_vol_ins_pur[['countyCode', 'propertyState', 'year', 'buildingType', 'sfha_flood_risk', 'bld_ins_per_pol', 'con_ins_per_pol']]
ins_rate_vol = ins_rate_vol[(ins_rate_vol.year==2021) & (ins_rate_vol.buildingType=='Residential')].reset_index(drop=True)
gb_ins_rate_vol = ins_rate_vol.groupby(['countyCode', 'sfha_flood_risk'])[['bld_ins_per_pol','con_ins_per_pol']].mean().reset_index()
gb_ins_rate_vol2 = pd.merge(left=gb_ins_rate_vol,
                            right=household_value[['CountyFIPS', 'County', 'State', 'county_state', 'Median_Value_of_Owner_Occupied_Units']],
                            left_on='countyCode',
                            right_on='CountyFIPS',
                            how='inner'
                            )  

c:\Users\pacha\Dropbox\datascience\projects\florita\notebooks\utils\utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column[1]] = [by_code[fips][column[0]][column[1]] for fips in FipsDF.CountyFIPS]
c:\Users\pacha\Dropbox\datascience\projects\florita\notebooks\utils\utils.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['County'] =  [by_code[fips]['CountyFull'] for fips in FipsDF.CountyFIPS]
c:\Users\pacha\Dropbox\datascience\projects\florita\notebooks\utils\utils.py:32: SettingWithCopyWa

In [18]:
# create function to calculate voluntarily purchased flood insurance
def vol_insurance(bld_ins_per_pol, con_ins_per_pol, sfha_flood_risk, Median_Value_of_Owner_Occupied_Units):
    if sfha_flood_risk == 1: # shfa=1 means building insurance is mandatory, just examine non_mandatory contents insurance coverage
        ins_rate = con_ins_per_pol/100000 # max fema contents coverage is $100000 per policy
    else:
        if con_ins_per_pol > 0: # sfha=0 and con_ins_per_pol>0 indicate voluntary purchases of building and contents insurance coverage
            if Median_Value_of_Owner_Occupied_Units < 250000: # max fema building coverage for single residential homes is $250000 per policy
                ins_rate = (bld_ins_per_pol + con_ins_per_pol)/(Median_Value_of_Owner_Occupied_Units+100000)
            else:
                ins_rate = (bld_ins_per_pol + con_ins_per_pol)/(250000+100000)
        else:
            if Median_Value_of_Owner_Occupied_Units < 250000:
                ins_rate = bld_ins_per_pol/Median_Value_of_Owner_Occupied_Units
            else:
                ins_rate = bld_ins_per_pol/250000
    return ins_rate*100

In [19]:
# apply function to relevant series
gb_ins_rate_vol2['vol_ins_rate'] = gb_ins_rate_vol2.apply(lambda x: vol_insurance(x.bld_ins_per_pol, 
                                                                                  x.con_ins_per_pol,
                                                                                  x.sfha_flood_risk, 
                                                                                  x.Median_Value_of_Owner_Occupied_Units
                                                                                  ),
                                                                                  axis=1
                                                                                  )

In [20]:
# collapsing data by county and saving output to csv
gb_ins_rate_vol3 = gb_ins_rate_vol2.groupby(['countyCode', 'County', 'State', 'county_state']).agg({
    'Median_Value_of_Owner_Occupied_Units': np.median,
    'vol_ins_rate': np.mean
    }).round(1).reset_index()
gb_ins_rate_vol3['Median_Value_of_Owner_Occupied_Units'] = gb_ins_rate_vol3['Median_Value_of_Owner_Occupied_Units'].astype(int)
gb_ins_rate_vol3.to_csv('../data/vol_ins_rates.csv')

In [30]:
# append county_state string to total claims data
# format total_claims column
fema_counts_total_avg = pd.read_csv('../data/fema_counts_total_avg.csv', dtype={'countyCode': str})
households = u.get_demographics([('Housing', 'Households')], name=True, state=True)
households['county_state'] = [c + ', ' + s if s=='AK' or s=='LA' else c + ' County, ' + s for c, s in zip(households.County, households.State)]
claims_tot = households.merge(right=fema_counts_total_avg,
                              left_on='CountyFIPS',
                              right_on='countyCode', 
                              how='inner'
                              )
claims_tot = claims_tot[['countyCode', 
                         'County', 
                         'State', 
                         'county_state', 
                         'total_claims'
                        ]]
claims_tot['total_claims'] = claims_tot['total_claims'].round(0)
claims_tot.loc[:, 'total_coverage'] = '$'+ claims_tot['total_coverage'].map('{:,.0f}'.format)
claims_tot.to_csv('../data/claims_tot.csv')
